# CSE 258: Assignment 1
### Benjamin Xia

### Setup

In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import feature_extraction
import random
from collections import defaultdict
from tqdm import tqdm
import gzip

import os

RANDOM_SEED = 0
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

### Preprocessing

#### Preprocess user/item ID's, compensation, early_access, and time

In [2]:
user_oe = preprocessing.OrdinalEncoder(dtype=np.int32, min_frequency=5, handle_unknown='use_encoded_value', unknown_value=6710)
item_oe = preprocessing.OrdinalEncoder(dtype=np.int32, min_frequency=5)

itemset = set() # Set of all unique users
userset = set() # Set of all unique items
U = defaultdict(set)
I = defaultdict(set)

ft = ['early_access', 'compensation'] # features unavailable/cannot be approximated in inference
def read_json(path):
    f: gzip.GzipFile = gzip.open(path)
    f.readline()
    for line in f:
        entry = eval(line)
        yield entry

# Encode userID and itemID as integers
def process_data():
    global itemset, userset, U, I
    data = []
    for entry in read_json('train.json.gz'):
        data.append(entry)

    df: pd.DataFrame = pd.DataFrame(data)
    del data
    itemset = set(df['gameID'].unique())
    userset = set(df['userID'].unique())

    U = dict(df.groupby('gameID')['userID'].unique())
    I = dict(df.groupby('userID')['gameID'].unique())
    U = { g : set(U[g]) for g in U }
    I = { u : set(I[u]) for u in I }

    df['userIDX'] = user_oe.fit_transform(df[['userID']])
    df['itemIDX'] = item_oe.fit_transform(df[['gameID']])
    df.rename({'gameID' : 'itemID'}, axis=1, inplace=True)


    df.drop(labels=['hours', 'user_id', 'date'], axis=1, inplace=True)


    # Get features that won't be available
    df.fillna(value=0, axis=1, inplace=True)
    df['compensation'] = df['compensation'].map(lambda x : x if x == 0 else 1)
    df[['early_access', 'compensation']] = df[['early_access', 'compensation']].astype(np.int32)

    time_label = df['hours_transformed']

    return df, time_label

df, time_label = process_data()
user_mean = df.groupby('userIDX')[ft].mean()
item_mean = df.groupby('itemIDX')[ft].mean()

In [4]:
df.drop(labels=ft + ['hours_transformed', 'found_funny'], axis=1, inplace=True)
df.head()

,userID,text,itemID,userIDX,itemIDX
0,u70666506,If you want to sit in queue for 10-20min and h...,g49368897,4740,1209
1,u18612571,I was really not a fan of the gameplay. Games ...,g73495588,1240,1800
2,u34283088,Vaas Montenegro is the reason why you should g...,g68047320,2314,1652
3,u16220374,"8/10 Wonderful game, simple controls and platf...",g51234623,1067,1244
4,u01499286,Never knew a guns had THAT many parts!,g25723374,92,609


#### Preprocess user text and convert to descriptors

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def get_text_embedding():
    if not os.path.isfile('./text_embed.npy'): # Generate new descriptors for each review using pretrained transformer
        dftext = df.groupby('itemIDX')['text'].apply(' '.join).reset_index()
        counter = feature_extraction.text.CountVectorizer(min_df=0.01, max_df=0.5, stop_words='english', max_features=2000, ngram_range=(1, 2), binary=True)
        wordcount = counter.fit_transform(dftext['text'])
        LDA = LatentDirichletAllocation(n_components=20, random_state=RANDOM_SEED)
        text_embed = LDA.fit_transform(wordcount)
        np.save('text_embed.npy', text_embed)
    else: # Text descriptors already computed
        text_embed = np.load('./text_embed.npy')

    return text_embed

text_embed = get_text_embedding()
text_embed = text_embed / np.linalg.norm(text_embed, axis=1)[...,None]

df.drop('text', axis=1, inplace=True)


In [6]:
df_played_train = df.iloc[:150000]
df_played_valid = df.iloc[150000:]

In [7]:
len(df.columns) + len(user_mean.columns) + len(item_mean.columns) + text_embed.shape[1]

28

#### Played dataset

In [8]:
torch.set_default_tensor_type(torch.DoubleTensor)
class PlayedDataset(Dataset):
    def __init__(self, df) -> None:
        super().__init__()
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        row = self.df.iloc[index]
        userID = row['userID']
        itemID = row['itemID']
        userIDX = row['userIDX']
        itemIDX = row['itemIDX']
        negaID = random.choice(tuple(itemset.difference(I[userID]))) # Negative item ID
        negaIDX = item_oe.transform([[negaID]])[0][0]
        # Build positive pair
        pos = np.concatenate((row[2:].to_numpy().astype(np.float32),
                              user_mean.iloc[userIDX].to_numpy().astype(np.float32),
                              item_mean.iloc[itemIDX].to_numpy().astype(np.float32),
                              text_embed[itemIDX].astype(np.float32)))
        negrow = row.copy()
        negrow['itemIDX'] = negaIDX
        neg = np.concatenate((negrow[2:].to_numpy().astype(np.float32),
                              user_mean.iloc[userIDX].to_numpy().astype(np.float32),
                              item_mean.iloc[negaIDX].to_numpy().astype(np.float32),
                              text_embed[negaIDX].astype(np.float32)))
        return torch.from_numpy(pos).to(dtype=torch.float64), torch.from_numpy(neg).to(dtype=torch.float64)

played_ds = PlayedDataset(df_played_train)
played_ds[0]

(tensor([4.7400e+03, 1.2090e+03, 2.1429e-01, 0.0000e+00, 5.7895e-01, 2.9240e-02,
         2.5428e-01, 6.9249e-01, 1.2759e-04, 1.2759e-04, 3.9793e-01, 1.2759e-04,
         1.2759e-04, 1.2759e-04, 3.9421e-01, 1.2759e-04, 1.2759e-04, 1.2759e-04,
         1.2759e-04, 3.7154e-01, 2.6741e-02, 1.9906e-02, 5.3868e-02, 1.2759e-04,
         1.2759e-04, 1.2759e-04]),
 tensor([4.7400e+03, 1.1900e+02, 2.1429e-01, 0.0000e+00, 0.0000e+00, 3.3898e-02,
         1.8379e-04, 4.6149e-01, 1.8379e-04, 1.5461e-01, 1.8379e-04, 1.8379e-04,
         1.8379e-04, 8.3567e-01, 2.5452e-01, 1.8379e-04, 1.8379e-04, 1.8379e-04,
         1.8379e-04, 1.8379e-04, 1.8379e-04, 1.8379e-04, 1.8379e-04, 1.8379e-04,
         1.8379e-04, 1.8379e-04]))

### Model Definition

In [9]:
class FactorizationMachine(nn.Module):
    def __init__(self, n_user, n_item, n_feature, latent_dim, weight=True) -> None:
        """
        n_user: Number of unique users
        n_item: Number of unique items
        n_feature: Number of extra features to use
        latent_dim: Dimension of latent representations of users/items/features
        """
        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.n_feature = n_feature
        self.latent_dim = latent_dim
        self.weight = weight
        self.user_latent = nn.Embedding(n_user, latent_dim)
        self.item_latent = nn.Embedding(n_item, latent_dim)
        if self.n_feature > 0:
            self.feat_latent = nn.Parameter(torch.randn(n_feature, latent_dim),requires_grad=True)
            self.feat_weight = nn.Linear(n_feature, 1)
        if self.weight:
            self.user_weight = nn.Embedding(n_user, 1)
            self.item_weight = nn.Embedding(n_item, 1)
        # "alpha" or "w_0" term will be absorbed into feat_weight linear's bias
    def forward(self, x) -> torch.Tensor:
        """
        Input shape: batch_size x (user idx, item idx, features) - 2 dimensional
        Returns: n x 1 tensor of predictions
        """
        if len(x.size()) == 1:
            x = x.unsqueeze(0)
        # f(u, i) = w_0 + \sum_{j=1}^{d} w_j * x_j
        out = torch.zeros((x.size()[0], 1), device=device)
        if self.n_feature > 0 and self.weight:
            out += self.feat_weight(x[:, 2:])
        users = x[:, 0].to(dtype=torch.int32)
        items = x[:, 1].to(dtype=torch.int32)
        if self.weight:
            out += self.user_weight(users)
            out += self.item_weight(items)
        # Nested summation thingy
        # Interactions between users/items and features
        u_embed = self.user_latent(users)
        i_embed = self.item_latent(items)
        out += (u_embed * i_embed).sum(dim=1).unsqueeze(-1)   # Dot product between user and item latent representations
        if self.n_feature > 0:
            # Interactions between features
            xfeature = x[:, 2:]

            out += ((u_embed @ self.feat_latent.T) * xfeature).sum(dim=1).unsqueeze(-1) # Dot product between user and feature latent representations
            out += ((i_embed @ self.feat_latent.T) * xfeature).sum(dim=1).unsqueeze(-1) # Dot product between item and feature latent representations
            out_1 = torch.matmul(xfeature, self.feat_latent).pow(2).sum(1, keepdim=True) #S_1^2
            out_2 = torch.matmul(xfeature.pow(2), self.feat_latent.pow(2)).sum(1, keepdim=True) # S_2
            out_inter = 0.5*(out_1 - out_2)
            out += out_inter
        return out


In [10]:
played_model = FactorizationMachine(len(df['userID'].unique()), len(df['itemID'].unique()), 24, 32, False).to(device) # 24 features

# if os.path.isfile('./played_model.pt'):
#     played_model.load_state_dict(torch.load('./played_model.pt'))
optimizer = optim.SGD(played_model.parameters(), lr=0.01)
def BPRLoss(pos_score, neg_score):
     return torch.sum(-(pos_score - neg_score).sigmoid().log())

In [11]:
batch_sz=20
print_iter=500
played_dl = DataLoader(dataset=played_ds,
                       batch_size=batch_sz,
                       shuffle=False, num_workers=2)
played_valid_ds = PlayedDataset(df_played_valid)
played_valid_dl = DataLoader(dataset=played_valid_ds,
                             batch_size=batch_sz,
                             num_workers=2)

In [12]:
def validate():
    with torch.no_grad():
        posscores = torch.zeros((0, 1), dtype=torch.float64, device=device)
        negscores = torch.zeros((0, 1), dtype=torch.float64, device=device)
        for i, (pos, neg) in tqdm(enumerate(played_valid_dl)):
            pos_score = played_model(pos.to(device))
            neg_score = played_model(neg.to(device))
            posscores = torch.concatenate((posscores, pos_score), dim=0)
            negscores = torch.concatenate((negscores, neg_score), dim=0)
        combined = torch.concatenate((posscores, negscores), dim=0)
        print(torch.sum(combined.argsort(dim=0)[len(played_valid_ds):] < len(played_valid_ds)) / (len(played_valid_ds)))
def epoch():
    running_loss = 0
    running_auc = 0
    for i, (pos, neg) in tqdm(enumerate(played_dl)):
        pos_score = played_model(pos.to(device))
        neg_score = played_model(neg.to(device))
        loss = BPRLoss(pos_score, neg_score)
        running_loss += loss
        running_auc += torch.sum(pos_score > neg_score) / batch_sz
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i == 0:
            print(f"{i}/{len(played_dl)}")
            print(f"loss: {running_loss}")
            print(f"AUC: {running_auc}")
        elif (i + 1) % print_iter == 0:
            print(f"{i}/{len(played_dl)}")
            print(f"loss: {running_loss / print_iter}")
            running_loss = 0
            print(f"AUC: {running_auc / print_iter}")
            running_auc = 0

In [ ]:
EPOCHS = 100

for i in range(EPOCHS):
    print(f"Epoch: {i} / {EPOCHS}")
    epoch()
    print(f'-----EPOCH {i} VALIDATION ACC %-----')
    validate()

Epoch: 0 / 100


6it [00:00, 10.17it/s]

0/7500
loss: 186.02142484397862
AUC: 0.6000000000000001


502it [00:23, 21.59it/s]

499/7500
loss: 130.07783733898788
AUC: 0.5156000000000004


...


4it [00:00, 15.29it/s]

0/7500
loss: 13.482288714394956
AUC: 0.8500000000000001


502it [00:23, 21.43it/s]

499/7500
loss: 22.638720264762537
AUC: 0.7298999999999998


972it [00:44, 21.88it/s]

In [ ]:
# 2 epochs - .625
with torch.no_grad():
    posscores = torch.zeros((0, 1), dtype=torch.float64, device=device)
    negscores = torch.zeros((0, 1), dtype=torch.float64, device=device)
    for i, (pos, neg) in tqdm(enumerate(played_dl)):
        pos_score = played_model(pos.to(device))
        neg_score = played_model(neg.to(device))
        posscores = torch.concatenate((posscores, pos_score), dim=0)
        negscores = torch.concatenate((negscores, neg_score), dim=0)
    combined = torch.concatenate((posscores, negscores), dim=0)
    print(torch.sum(combined.argsort(dim=0)[len(played_valid_ds):] < len(played_valid_ds)) / (len(played_valid_ds)))

In [ ]:
cnt = 0
for i in range(1000):
    pair = played_valid_ds[100]
    p1 = played_model(pair[0].to(device))
    p2 = played_model(pair[1].to(device))
    if p1 > p2:
        cnt += 1
print(cnt / 1000)

In [ ]:
test = pd.read_csv('./pairs_Played.csv')
test['itemID'] = test['gameID']
# Map unseen entries to default user (this user is already grouped with other users due to their few # of reviews in training set)
test['userID'] = test['userID'].map(lambda x: x if x in userset else 'u03473346')
test['userIDX'] = user_oe.transform(test[['userID']])
test['itemIDX'] = item_oe.transform(test[['gameID']])
test.drop(columns=['gameID', 'prediction'], inplace=True)

In [ ]:
test.head()

In [ ]:
played_test_ds = PlayedDataset(test)
played_test_dl = DataLoader(dataset=played_test_ds,
                           batch_size=len(played_test_ds),
                           shuffle=True, num_workers=1)

In [ ]:
preds = 0
for pos, neg in played_test_dl:
    preds = model(pos.to(device))

In [ ]:
validate()

In [ ]:
played_model(played_ds[0][0].to(device))

In [ ]:
played_model(played_ds[0][0].to(device))

In [ ]:
played_model(played_ds[0][1].to(device))

In [ ]:
played_model(played_ds[0][1].to(device))

In [ ]:
torch.save(played_model.state_dict(), './played_model.pt')